<a href="https://colab.research.google.com/github/dzeko5959/AI/blob/main/ML/UDEM/P_P1_Regresi%C3%B3n_Seunghyeon_Lee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Import and Visualize Data**

Using the "Global Traffic Accidents Dataset" from Kaggle.

In [ ]:
import kagglehub

# Global Traffic Accidents Dataset
path = kagglehub.dataset_download("adilshamim8/global-traffic-accidents-dataset")

print("Path to dataset files:", path)

100%|██████████| 292k/292k [00:00<00:00, 45.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/adilshamim8/global-traffic-accidents-dataset/versions/1


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving global_traffic_accidents.csv to global_traffic_accidents.csv


In [ ]:
import pandas as pd

df = pd.read_csv("global_traffic_accidents.csv")
df.head()
df.describe()
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Accident ID        10000 non-null  object 
 1   Date               10000 non-null  object 
 2   Time               10000 non-null  object 
 3   Location           10000 non-null  object 
 4   Latitude           10000 non-null  float64
 5   Longitude          10000 non-null  float64
 6   Weather Condition  10000 non-null  object 
 7   Road Condition     10000 non-null  object 
 8   Vehicles Involved  10000 non-null  int64  
 9   Casualties         10000 non-null  int64  
 10  Cause              10000 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 859.5+ KB


,0
Accident ID,0
Date,0
Time,0
Location,0
Latitude,0
Longitude,0
Weather Condition,0
Road Condition,0
Vehicles Involved,0
Casualties,0


**2. Data preprocessing**

Fill in missing values by average, and convert 'Date' and 'Time' variables to datetime format.

In [ ]:
df['Latitude'].fillna(df['Latitude'].mean(), inplace=True)
df['Longitude'].fillna(df['Longitude'].mean(), inplace=True)
df['Weather Condition'].fillna('Unknown', inplace=True)
df['Road Condition'].fillna('Unknown', inplace=True)
df['Casualties'].fillna(df['Casualties'].mean(), inplace=True)

df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

print(df.head())

  Accident ID       Date      Time           Location   Latitude   Longitude  \
0    b0dd6f57 2023-04-19  06:39:00      Mumbai, India  13.488432  -73.290682   
1    debfad09 2023-01-17  02:47:00  São Paulo, Brazil -37.798317  -32.244242   
2    6d69aa36 2024-04-09  02:55:00  Sydney, Australia  33.767869  104.869018   
3    425bb1f0 2023-10-10  11:23:00       Tokyo, Japan  -0.378031 -165.825855   
4    90d5cf62 2023-01-02  12:07:00     Beijing, China  41.254879  -30.776959   

  Weather Condition Road Condition  Vehicles Involved  Casualties  \
0              Snow          Snowy                  5           7   
1             Clear            Icy                  4           1   
2              Rain          Snowy                  1           7   
3             Storm            Wet                  4           0   
4             Storm          Snowy                  3           9   

              Cause  
0  Reckless Driving  
1     Drunk Driving  
2  Reckless Driving  
3     Drunk Driv

<ipython-input-24-8cf56a679cd1>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Latitude'].fillna(df['Latitude'].mean(), inplace=True)
<ipython-input-24-8cf56a679cd1>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

**3. Create dummy variables for categorical variables**

Convert categorical variables such as Weather Condition, Road Condition, Location and Cause to dummy variables.

In [ ]:
df = pd.get_dummies(df, columns=['Weather Condition', 'Road Condition', 'Location', 'Cause'], drop_first=True)

print(df.head())

  Accident ID       Date      Time   Latitude   Longitude  Vehicles Involved  \
0    b0dd6f57 2023-04-19  06:39:00  13.488432  -73.290682                  5   
1    debfad09 2023-01-17  02:47:00 -37.798317  -32.244242                  4   
2    6d69aa36 2024-04-09  02:55:00  33.767869  104.869018                  1   
3    425bb1f0 2023-10-10  11:23:00  -0.378031 -165.825855                  4   
4    90d5cf62 2023-01-02  12:07:00  41.254879  -30.776959                  3   

   Casualties  Weather Condition_Fog  Weather Condition_Hail  \
0           7                  False                   False   
1           1                  False                   False   
2           7                  False                   False   
3           0                  False                   False   
4           9                  False                   False   

   Weather Condition_Rain  ...  Location_Paris, France  \
0                   False  ...                   False   
1                 

**4. Feature Selection**

*   Unnecessary variables : 'Accident ID', 'Date', 'Time'
*   Output value from model Y : 'Casualties'

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

X = df.drop(['Accident ID', 'Date', 'Time', 'Casualties'], axis=1)
y = df['Casualties']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Select features with Lasso regression
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

selected_features = X.columns[lasso.coef_ != 0]
print("Selected Features:", selected_features)

Selected Features: Index(['Latitude', 'Longitude'], dtype='object')


**5. Create a regression model**

We generate a linear regression model, predict it, and evaluate its performance.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train[selected_features], y_train)

y_pred = lr_model.predict(X_test[selected_features])

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 10.030446598824621
R-squared: -0.0021471369066485835


In [ ]:
intercept = lr_model.intercept_
coefficients = lr_model.coef_
feature_names = selected_features

equation = f"y = {intercept:.4f}"

for coef, feature in zip(coefficients, feature_names):
    equation += f" + ({coef:.4f}) * {feature}"

print("\nLinear Regression Equation:")
print(equation)


Linear Regression Equation:
y = 5.0196 + (-0.0005) * Latitude + (-0.0003) * Longitude


**6. Model evaluation**

Evaluate the significance of the regression coefficient through p-value calculation.

In [ ]:
print("Coefficients:", lr_model.coef_)

import statsmodels.api as sm
X_train_selected = sm.add_constant(X_train[selected_features])
model = sm.OLS(y_train, X_train_selected).fit()

print(model.summary())

Coefficients: [-0.00049139 -0.00026854]
                            OLS Regression Results                            
Dep. Variable:             Casualties   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5846
Date:                Thu, 20 Feb 2025   Prob (F-statistic):              0.557
Time:                        17:54:36   Log-Likelihood:                -20455.
No. Observations:                8000   AIC:                         4.092e+04
Df Residuals:                    7997   BIC:                         4.094e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       

**7. Result Interpretation**

*   10.03 indicates that the forecast has some error when compared to the actual value.
*   -0.0021 indicates that the model is not describing the data well at all; this value is very low, indicating that the regression model did not capture any significant patterns in the data.
*   The intercept appears to be about 5.02.
*   The coefficient of the latitude variable was found to be -0.00049139. This means that when the latitude increases, accident damage tends to decrease slightly. However, this coefficient is not statistically significant as the p-value is very large at 0.466.
*   The coefficient of the hardness variable is -0.00026854. This indicates that when hardness increases, accident damage tends to decrease slightly. Likewise, the p-value is 0.428, which is not statistically significant.
*   The p-values are very large, 0.466 (latitude) and 0.428 (hardness), respectively. This means that the relationship between both variables and the dependent variable Casualties in the model is not statistically significant.
*   The F-statistic and corresponding p-values indicate that the model is not statistically significant. Since the p-value is large at 0.557, we conclude that the current model is not overall significant.

**8. Inference**

This model is ineffective in explaining the relationship between Casualties and latitude and longitude. MSE and R² values confirm that the model is not predicting well, and we show that even the p-values of each independent variable (latitude and longitude) do not show a significant relationship in the model.